In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [72]:
import pickle
import threading
import numpy as np
from codes.models import Model
from codes.models.ExpGlm import ExpGlm
from codes.models.WblGlm import WblGlm
from codes.models.NpGlm import NpGlm
from codes.models.RayGlm import RayGlm
from codes.features.delicious.extraction import run as delicious_run
from codes.features.utils import timestamp_delta_generator
from codes.features.autoencoder import encode
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [4]:
np.random.seed(0)

In [49]:
def get_model(dist):
    return {
        'np': NpGlm(),
        'wbl': WblGlm(),
        'exp': ExpGlm(),
        'ray': RayGlm(),
        'pow': PowGlm(),
        'gom': GomGlm()
    }[dist]

In [6]:
def generate_c_index(T_true, T_pred, Y):
    total_number_of_pairs = 0
    number_of_correct_predictions = 0

    for i in range(len(T_true)):
        for j in range(len(T_true) - 1, i, -1):
            if Y[i] != 0 or Y[j] != 0:  # if one or both of the samples are in observation window
                total_number_of_pairs += 1
                if T_true[i] > T_true[j] and T_pred[i] > T_pred[j]:
                    number_of_correct_predictions += 1
                if T_true[i] < T_true[j] and T_pred[i] < T_pred[j]:
                    number_of_correct_predictions += 1
                if T_true[i] == T_true[j] and T_pred[i] == T_pred[j]:
                    number_of_correct_predictions += 1

    return number_of_correct_predictions / total_number_of_pairs

In [7]:
def prepare_data(X, Y, T):
    T = T.astype(np.float64)
    T /= timestamp_delta_generator(months=1)
    T += np.random.rand(len(T)) * Y

    index = np.argsort(T, axis=0).ravel()
    X = X[index, :]
    Y = Y[index]
    T = T[index]

    return X, Y, T

In [71]:
def evaluate(model: Model, X_train: np.ndarray, Y_train: np.ndarray, T_train: np.ndarray, X_test: np.ndarray,
             Y_test: np.ndarray, T_test: np.ndarray):
    model.fit(X_train, Y_train, T_train)

    # T_pred = model.mean(X_test)
    T_pred = model.quantile(X_test, .5).ravel()
    T_pred = np.fmin(T_pred, max(T_test))

    c_index = generate_c_index(T_test, T_pred, Y_test)

    k = Y_test.sum()
    # X_test = X_test[:k, :]
    T_test = T_test[:k]
    T_pred = T_pred[:k]

#     threshold = [.5, 1, 1.5, 2, 2.5, 3]

    # lb1 = model.quantile(X_test, .25).ravel()
    # ub1 = model.quantile(X_test, .75).ravel()
    #
    # lb2 = model.quantile(X_test, .2).ravel()
    # ub2 = model.quantile(X_test, .8).ravel()
    #
    # lb3 = model.quantile(X_test, .15).ravel()
    # ub3 = model.quantile(X_test, .85).ravel()
    #
    # C1 = np.logical_and(lb1 <= T_test, T_test <= ub1)
    # C2 = np.logical_and(lb2 <= T_test, T_test <= ub2)
    # C3 = np.logical_and(lb3 <= T_test, T_test <= ub3)
    #

#     distance = np.zeros((len(threshold)))
#     for i in range(len(threshold)):
#         distance[i] = (res <= threshold[i]).sum() / len(res)

#     ev = explained_variance_score(T_test, T_pred)
    mae = mean_absolute_error(T_test, T_pred)
    rmse = mean_squared_error(T_test, T_pred)**.5
    msle = mean_squared_log_error(T_test, T_pred)
    mad = median_absolute_error(T_test, T_pred)
#     r2 = r2_score(T_test, T_pred)
    # ci5 = C1.sum() / len(C1)
    # ci6 = C2.sum() / len(C2)
    # ci7 = C3.sum() / len(C3)

    return mae, rmse, msle, mad, c_index #, distance

In [81]:
def cross_validate(dists, X_stat, X, Y, T, cv=5):
    threads = []
    results = {dist+pos: [] for dist in dists for pos in ['', '_stat']}
    k_fold = StratifiedKFold(n_splits=cv, shuffle=True)

    for training_indices, test_indices in k_fold.split(X=X, y=Y):
        X_stat_train = X_stat[training_indices, :]
        X_train = X[training_indices, :]
        Y_train = Y[training_indices]
        T_train = T[training_indices]

        X_stat_test = X_stat[test_indices, :]
        X_test = X[test_indices, :]
        Y_test = Y[test_indices]
        T_test = T[test_indices]

        def worker():
            for dist in dists:
                model = get_model(dist)
                scores = evaluate(model, X_train, Y_train, T_train, X_test, Y_test, T_test)
                results[dist].append(scores)
                scores_stat = evaluate(model, X_stat_train, Y_train, T_train, X_stat_test, Y_test, T_test)
                results[dist+'_stat'].append(scores_stat)

        job = threading.Thread(target=worker)
        threads.append(job)

    for t in threads:
        t.start()
    for t in threads:
        t.join()
    
    return results

In [47]:
def get_name(dist):
    return {
        'np': 'NP-Glm',
        'wbl': 'Wbl-Glm',
        'exp': 'Exp-Glm',
        'ray': 'Ray-Glm',
        'gom': 'Gom-Glm'
    }[dist]

In [38]:
X_list, Y_raw, T_raw = delicious_run(delta=3, observation_window=12, n_snapshots=9)

16:10:36: generating indexer ...
16:10:37: parsing dataset ...


2008-12-25 00:00:00


16:10:37: generating samples ...
16:10:37: extracting ...


2596


16:10:38: parsing dataset ...


...

16:10:44: extracting ...
16:10:44: parsing dataset ...


2006-12-12 00:00:00


16:10:45: extracting ...
16:10:45: parsing dataset ...


2006-09-10 01:00:00


16:10:46: extracting ...
16:10:46: done.


In [43]:
X_raw = encode(X_list, Y_raw, epochs=100, latent_factor=2)

Epoch 1/100
2596/2596 [==============================] - 2s 933us/step - loss: 5.7325
Epoch 2/100
2596/2596 [==============================] - 1s 227us/step - loss: 5.0457
Epoch 3/100
2596/2596 [==============================] - 1s 197us/step - loss: 4.8600
Epoch 4/100
2596/2596 [==============================] - 1s 204us/step - loss: 4.5557
Epoch 5/100
2596/2596 [==============================] - 1s 200us/step - loss: 3.4315
Epoch 6/100
2596/2596 [==============================] - 1s 218us/step - loss: 3.3399
Epoch 7/100
2596/2596 [==============================] - 1s 206us/step - loss: 3.2790
Epoch 8/100
2596/2596 [==============================] - 1s 205us/step - loss: 3.2311
Epoch 9/100
2596/2596 [==============================] - 1s 209us/step - loss: 3.1953
Epoch 10/100
2596/2596 [==============================] - 1s 212us/step - loss: 3.1715
Epoch 11/100
2596/2596 [==============================] - 1s 242us/step - loss: 3.1579
Epoch 12/100
2596/2596 [============================

2596/2596 [==============================] - 1s 214us/step - loss: 1.8879
Epoch 96/100
2596/2596 [==============================] - 1s 214us/step - loss: 1.8872
Epoch 97/100
2596/2596 [==============================] - 1s 274us/step - loss: 1.8862
Epoch 98/100
2596/2596 [==============================] - 1s 217us/step - loss: 1.8851
Epoch 99/100
2596/2596 [==============================] - 1s 226us/step - loss: 1.8840
Epoch 100/100
2596/2596 [==============================] - 1s 209us/step - loss: 1.8828


In [80]:
print(len(T))
X, Y, T = prepare_data(X_raw, Y_raw, T_raw)
scaler = MinMaxScaler(copy=True)
X_stat = scaler.fit_transform(X_list[-1])

dists = [
    'np',
    'wbl',
#     'exp',
#     'ray',
    # 'gom'
]

results = cross_validate(dists, X_stat, X, Y, T, cv=5)
#         ev, mae, rmse, msle, mad, r2, c_index #, distance
for pos in ['', '_stat']:
    for dist in dists:
        print(get_name(dist)+pos)
        result = np.array(results[dist+pos])
        mean = result.mean(axis=0)
    #     std = result.std(axis=0)
        print('MAE=%.2f\tRMSE=%.2f\tMSLE=%.2f\tMAD=%.2f\tCI=%.2f' % tuple(mean))

2596
NP-Glm
MAE=5.00	RMSE=6.14	MSLE=0.89	MAD=4.36	CI=0.69
Wbl-Glm
MAE=5.07	RMSE=6.23	MSLE=0.89	MAD=4.15	CI=0.58
NP-Glm_stat
MAE=5.46	RMSE=6.48	MSLE=1.10	MAD=5.40	CI=0.63
Wbl-Glm_stat
MAE=5.56	RMSE=6.56	MSLE=1.10	MAD=4.93	CI=0.54
